The import statements and much of the code in this notebook are sourced from the script in the original LGN repository.

In [7]:
import sys, os, time
import pandas as pd
import h5py as h5
import numpy as np
from numba import jit
import re

In [8]:
@jit
def pt(momentum):
    return np.sqrt(np.dot(momentum[1:3],momentum[1:3]))

In [9]:
# dot.py
# Author: Jan Offermann
# Date: 03/19/20

# Works for N+1 dim. Minkowski space, w/ metric signature (+,-,-,...,-).

# Given two 4-momenta, return the dot product.
@jit
def dot(p1,p2):
    return p1[0] * p2[0] - np.dot(p1[1:],p2[1:])

# Given two (N,1) lists of 4-momenta, return list of (N) dot products.
@jit
def dots(p1s,p2s):
    return np.array([dot(p1s[i],p2s[i]) for i in range(p1s.shape[0])])

# Given one (N,1) list of 4-momenta, return list of (N) norms.
@jit
def masses(p1):
    return np.sqrt(np.maximum(0.,dots(p1,p1)))

# Given two (N,1) lists of 4-momenta, return array (N,N) of dot products.
# Here, position (i,j) is the dot product of the i'th element of the first
# list and the j'th element of the second list.
@jit
def dots_matrix_multi(p1s,p2s):
    a = p1s.shape[0]
    b = p2s.shape[0]
    return np.array([[dot(p1s[i],p2s[j]) for j in range(b)] for i in range(a)])

# Given one (N,1) lists of 4-momenta, return array (N,N) of dot products.
# Here, position (i,j) is the dot product of the i'th element of the list
# and the j'th element of the list.
@jit
def dots_matrix_single(p1s):
    a = p1s.shape[0]
    matrix = np.zeros((a,a),dtype=np.dtype('f8'))
    for i in range(a):
        for j in range(i+1): # get the diagonal elements
            matrix[i,j] = dot(p1s[i],p1s[j])
            matrix[j,i] = matrix[i,j] # symmetric
    return matrix
    
# Handler for dots_matrix_single & dots_matrix_multi.
# The former is much faster, and should be used if
# p1s == p2s. It may still be faster to directly call
# the underlying functions, since it avoids this
# (somewhat costly?) check on array equality.
def dots_matrix(p1s,p2s):
    if(np.array_equal(p1s,p2s)):
        return dots_matrix_single(p1s)
    else:
        return dots_matrix_multi(p1s,p2s)

def test(N = 1000):
    import time
    p1s = np.random.rand(N,8,4)
    p2s = np.random.rand(N,8,4)
    start = time.time()
    for i in range(N):
        a = dots_matrix(p1s[i],p2s[i])
    end = time.time()
    print('t1 = ', end-start)
    start = time.time()
    for i in range(N):
        a = dots_matrix(p1s[i],p1s[i])
    end = time.time()
    print('t2 = ', end-start)
    return


In [10]:
# Simplify filenames for readability.
# This might not work well with some corner cases,
# needs review for robustness.
def SimplifyPath(path):
    path = re.sub(r'/[^/]*/\.\./', r'/', path)
    if path is re.sub(r'/[^/]*/\.\./', r'/', path):
        return path
    else:
        return SimplifyPath(path)


In [11]:
def convert(file_to_convert, add_beams = False, dot_products = False, double_precision = True):
    # Setup beam particles to be added, if required.
    # If added, they will go in the last 2 positions
    # of the 4-momentum list data['Pmu'].
    nbeam = 0
    if(add_beams): nbeam = 2
    beam_mass = 0. # mass for beam particles
    beam_pz = 1.
    beam_E = np.sqrt(beam_mass * beam_mass + beam_pz * beam_pz)
    beam_vecs = np.array([beam_E,0.,0.,beam_pz])
    beam_vecs = np.array([beam_vecs,beam_vecs],dtype=np.dtype('f8'))
    beam_vecs[-1,-1] = -1. * beam_vecs[-1,-1]
    
    # Number of 4-momenta per event, and number of columns in the raw file's DataFrame.
    nvectors_original = 200
    nvectors = nvectors_original + nbeam # 200 4-momenta per event in data
    ncolumns = 806 # expected number of columns in toptag dataset
    
    # Get the DataFrame.
    frame = pd.read_hdf(file_to_convert, 'table')
    nentries = frame.shape[0] # number of entries to convert
    if(frame.shape[1] != ncolumns): # Check that number of columns matches what is expected
        print('Warning: Expected ' + str(ncolumns) +' columns in ' + file_to_convert + ', found ' + str(frame.shape[1]) + '.')
        return
    
    precision = 'f8' #64-bit floating-point number (default)
    if(not double_precision): precision = 'f4' #32-bit floating-point number
    
    # Dictionary holding the data.
    data = {'Nobj':np.zeros(nentries,np.dtype('i2')), # number of 4-momenta per event
    'Pmu':np.zeros((nentries,nvectors,4),np.dtype(precision)), # list of 4-momenta for each event
    'truth_Pmu':np.zeros((nentries,4),np.dtype(precision)), # top 4-momentum for each event (only meaningful for signal)
    'is_signal':np.zeros(nentries,np.dtype('i2')), # signal/background flag
    'jet_pt':np.zeros(nentries,np.dtype(precision)), # jet pt -- used for splitting dataset, *not* used by network
    'label':np.zeros((nentries,nvectors),np.dtype('i2')), # Lorentz-inv. labels -- used to identify different types of particles, e.g. beam vs. non-beam
    'mass':np.zeros((nentries,nvectors),np.dtype(precision)) # particle masses
    }
    
    # Add column for dot products, if required.
    if(dot_products):
        # position (i,j,k) gives dot product p_j p^k, for event i
        data['dots'] = np.zeros((nentries,nvectors,nvectors),np.dtype(precision))
    
    # 4-vectors occupy columns 0-799.
    # truth 4-vector occupues columns 800-803.
    # ttv is in column 804 (redundant variable).
    # is_signal in column 805.
    
    # Get the indices in the raw file corresponding to the reco particle momenta, and the truth-level top momentum.
    pmu_idxs = np.array([np.linspace(0,nvectors_original*4,nvectors_original,False) + x for x in range(4)])
    truth_pmu_idxs = np.linspace(4 * nvectors_original,4 * nvectors_original+4,4,False)

    # Loop over entries, fill the numpy arrays in memory
    for i in range(nentries):
        # Fill in 4-momenta of the reco particles
        for j in range(4): data['Pmu'][i,:nvectors_original,j] = frame.iloc[i,pmu_idxs[j]].values[:] # 4-momenta from the file
        
        # Find + fill in number of non-zero reco particles
        nobj = np.nonzero(frame.iloc[i,pmu_idxs[0]].values == 0.)[0]
        if(nobj.shape[0] == 0): nobj = nvectors # no zeroes found -> (E_0...E_199) must all be non-zero
        else: nobj = nobj[0] + nbeam # Must add nbeam here, in case it is non-zero.
        data['Nobj'][i] = nobj
        
        # Signal flag
        data['is_signal'][i] = frame.iloc[i,-1]
        
        # Fill in truth particle
        data['truth_Pmu'][i,:] = frame.iloc[i,truth_pmu_idxs].values[:]
        
        # Jet pT, from reco particles
        data['jet_pt'][i] = pt(np.sum(data['Pmu'][i,:,:],axis=0))
        
        # Now, add beam particles if necessary.
        if(add_beams): data['Pmu'][i,[-2,-1],:] = beam_vecs
        
        # Now, fill in the particle labels.
            #  1: reco particle
            #  0: no particle (empty)
            # -1: beam particle
        data['label'][i,:nobj] = 1 # default entries are zero
        if(add_beams): data['label'][i,-2:]=-1
        
        # Fill in the particle masses.
        data['mass'][i,:] = masses(data['Pmu'][i,:,:])
        
        # Now, fill in the dot products if necessary.
        if(dot_products): data['dots'][i,:,:] = dots_matrix_single(data['Pmu'][i])
            
    # numpy arrays are filled, now we must write to a new file
    # Prepare the output filename
    # output_filename = os.path.dirname(os.path.realpath(__file__)) + '/' + file_to_convert.split('/')[-1]
    # output_filename = output_filename.replace('.h5','_c.h5')
    output_filename = "train_c.h5"
    
    # mildly paranoid safety to prevent overwrite of raw info
    # if((output_filename is file_to_convert) or ('.h5' not in output_filename)):
        # import uuid
        # output_filename = os.path.dirname(os.path.realpath(__file__)) + '/' + str(uuid.uuid4().hex) + '.h5'

    # Determine which data columns to write
    keys_to_write = list(data.keys()) # TODO: For now, we will always write all columns.
#    if(dot_products): # "Pmu" should not be written in this case, not needed for this use case
#        keys_to_write = [x for x in keys_to_write if x!='Pmu']
    with h5.File(output_filename, 'w') as f:
        [f.create_dataset(key, data=data[key],compression='gzip') for key in keys_to_write]
    print('File saved as ' + output_filename)
    return

Make sure for each file that you run to change the `output_filename` in the cell above. Otherwise, run the cell below with the proper `file_to_convert` for train, test, and val. Converting the train dataset took about two hours on my machine, and converting the test and validation datasets took a bit under one hour each.

In [12]:
file_to_convert = "zenodo/" + "train" + ".h5"

add_beams = False
dot_products = False
double_precision = True

if(add_beams): print('Adding beam particles.')
if(dot_products): print('Adding dot products of the 4-momenta to \'dots\' data column.')
if(not double_precision): print('Using 32-bit floating point precision.')

start = time.time()
output_filename = ''
convert(file_to_convert, add_beams, dot_products, double_precision) # Performs file conversion
end = time.time()
elapsed = end - start
print('Time elapsed: ' + str(elapsed) + '.')

File saved as train_c.h5
Time elapsed: 6597.39857172966.
